In [1]:
from src import NeuralTester, ExperimentConfig
import torch
from src.learner import (
    PymooLearner,
    RevDELearner,
    GeneticLearner,
    PYMOO_DE_DEFAULT_PARAMS,
    REV_DE_DEFAULT_PARAMS,
    GENETIC_DEFAULT_PARAMS,
)
from src.objective_functions import get_penalized_distance, ms_ssim, ssim_d2, uqi
from models import load_stylegan
import numpy as np
import logging
import wandb
import sys
from functools import partial
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.operators.sampling.lhs import LHS

## Example usage of the Neural Tester
We use wandb to log progress in various steps!

In [2]:
wandb.login()

wandb: Currently logged in as: oliverweissl. Use `wandb login --relogin` to force relogin


True

In [3]:
# Setup logging to be able to view logging outputs in notebooks.
logging.basicConfig(
    level=logging.INFO, format="[%(asctime)s - %(name)s - %(levelname)s] %(message)s"
)
logger = logging.getLogger()
logger.handlers[0].stream = sys.stdout

In [4]:
# Define the configurations for our experiments.
conf = ExperimentConfig(
    samples_per_class=10,
    generations=50,
    mix_dim_range=(0, 8),
    predictor="../models/wrn_mnist.pkl",
    generator="../models/sg2_mnist.pkl",
    learner=PymooLearner,
    metric=uqi,
)
learner_params = PYMOO_DE_DEFAULT_PARAMS

learner_params["n_var"] = conf.genome_size  # for pymoo
learner_params["algo_params"]["pop_size"] = 10 * conf.genome_size
# learner_params["x0"] = np.random.rand(learner_params["population_size"] * 2, conf.genome_size)  # for own

In [5]:
predictor = torch.load(conf.predictor)  # The System under test (SUT)
generator = load_stylegan(conf.generator)  # The generator network (a stylegan in this case)
learner = conf.learner(**learner_params)  # The learner for search based optimization of candidates.
objective_function = partial(
    get_penalized_distance, metric=conf.metric
)  # The objective function to calculate fitness with.
device = torch.device("cuda")  # The target device for all operaitons.

In [6]:
tester = NeuralTester(
    predictor=predictor,
    generator=generator,
    learner=learner,
    objective_function=objective_function,
    num_generations=conf.generations,
    mix_dim_range=conf.mix_dim_range,
    device=device,
)  # Here we initialize the Tester object.

In [ ]:
tester.test(
    num_classes=10,
    samples_per_class=conf.samples_per_class,
)  # We start the testing procedure.

[2024-11-07 16:40:18,592 - root - INFO] Start testing. Number of classes: 10, iterations per class: 10, total iterations: 100



[2024-11-07 16:40:20,042 - root - INFO] Generate seed(s) for class: 0.
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
[2024-11-07 16:40:20,619 - root - INFO] 	Found 1 valid seed(s) after: 1 iterations.
[2024-11-07 16:40:20,943 - root - INFO] Running Search-Algorithm for 50 generations.
